# GEMM on GPU

## 1. Set-up 

In [2]:
# Mount google drive 
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Make sure your token is stored in a txt file at the location below.
# This way there is no risk that you will push it to your repo
# Never share your token with anyone, it is basically your github password!
with open('/content/gdrive/MyDrive/ece5545/token.txt') as f:
    token = f.readline().strip()
# Use another file to store your github username    
with open('/content/gdrive/MyDrive/ece5545/user.txt') as f:
    handle = f.readline().strip()

In [3]:
# Clone your github repo
YOUR_TOKEN = token
YOUR_HANDLE = handle
BRANCH = "main"

%mkdir /content/gdrive/MyDrive/ece5545
%cd /content/gdrive/MyDrive/ece5545
!git clone https://{YOUR_TOKEN}@github.com/ML-HW-SYS/a3-{YOUR_HANDLE}.git
%cd /content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}
!git checkout {BRANCH}
!git pull
%cd /content/gdrive/MyDrive/ece5545

PROJECT_ROOT = f"/content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}"

mkdir: cannot create directory ‘/content/gdrive/MyDrive/ece5545’: File exists
/content/gdrive/MyDrive/ece5545
fatal: destination path 'a3-ethanglaser' already exists and is not an empty directory.
/content/gdrive/MyDrive/ece5545/a3-ethanglaser
M	1-conv1d_cpu.ipynb
M	2-conv1d_gpu.ipynb
M	5-conv2d_dw_gpu.ipynb
Already on 'main'
Your branch is up to date with 'origin/main'.
Already up to date.
/content/gdrive/MyDrive/ece5545


In [4]:
# This extension reloads all imports before running each cell
%load_ext autoreload
%autoreload 2

In [5]:
!ls {PROJECT_ROOT}

1-conv1d_cpu.ipynb   4-gemm_gpu.ipynb	    README.md
2-conv1d_gpu.ipynb   5-conv2d_dw_gpu.ipynb  src
3-conv1d_fpga.ipynb  leaderboard_id.txt     tests


## 2. Install TVM

In [6]:
!pip install tlcpack-nightly-cu102 -f https://tlcpack.ai/wheels

Looking in links: https://tlcpack.ai/wheels
     |████████████████████████████████| 402.4 MB 18 kB/s 


## 3. Implement `make_conv1d_gpu_scheduler_func` function in `src.ops`

In that function, you are required to implemented 1D convolution and use TVM to optimize it.
Let $x \in \mathbb{R}^m$ and $y \in \mathbb{R}^n$, then 
$$
\operatorname{conv1d}(x, y)_i = \sum_{j=-\infty}^{\infty} x[j]y[i-j], \forall i \in \{0, 1, \dots, m + n - 1\}
$$

Please use zero padding and unit stride. Please see the numpy convolution function for more detail: [link](https://numpy.org/doc/stable/reference/generated/numpy.convolve.html).

The `make_conv1d_gpu_scheduler_func` takes $m$ and $n$, which are the size of the two 1D input array. 
You should return both the TVM scheduler and the TVM opterator for 
1. Input $x$
2. Input $y$
3. Output $out$

The scheduler should be able to used to build a function with signature $func(x, y, out)$. 
Please see the following cells for usage.

In [9]:
import tvm
import numpy as np
import sys
# Adding assignment 3 to the system path
# Make sure this matches your git directory
sys.path.insert(0, PROJECT_ROOT)
from src.ops import make_gemm_gpu_scheduler

M = 1024
N = 512
K = 2048
dtype = 'float32'
a_np = np.random.rand(M, K).astype(dtype)
w_np = np.random.rand(K, N).astype(dtype)
b_np = np.matmul(a_np, w_np)

s, A, W, B = make_gemm_gpu_scheduler(M, K, N) 
func = tvm.build(s, [A, W, B], "cuda")

dev = tvm.cuda(0)
a = tvm.nd.array(a_np, dev)
w = tvm.nd.array(w_np, dev)
b = tvm.nd.array(np.zeros((M, N), dtype), dev)
func(a, w, b)
evaluator = func.time_evaluator(func.entry_name, dev, number=1, repeat =1)


print("Answer:", b_np)
print("Output:", b)
print(f"1DConv TVM: %f ms" % (evaluator(a, w, b).mean * 1e3))

Answer: [[515.6487  535.2116  505.88492 ... 524.45825 516.8844  512.7164 ]
 [498.18146 523.27893 496.16766 ... 505.30643 503.39352 506.6748 ]
 [510.11017 529.15594 500.88147 ... 517.6482  511.4097  506.4373 ]
 ...
 [517.9055  532.4307  511.58652 ... 530.3432  519.29584 518.61475]
 [506.67484 534.5979  507.48535 ... 528.90424 520.9872  517.5062 ]
 [518.7657  533.7033  497.42673 ... 515.9833  523.2933  514.14545]]
Output: [[515.64856 535.2114  505.88403 ... 524.45844 516.8849  512.7166 ]
 [498.18152 523.27875 496.16754 ... 505.30646 503.39325 506.67508]
 [510.11063 529.1561  500.88153 ... 517.64856 511.40994 506.43732]
 ...
 [517.9059  532.43066 511.58694 ... 530.34314 519.2962  518.61475]
 [506.6748  534.5976  507.4848  ... 528.9038  520.98773 517.50574]
 [518.76544 533.70337 497.42636 ... 515.9832  523.29315 514.14526]]
1DConv TVM: 10.174799 ms


In [8]:
print(tvm.lower(s, [A, W, B], simple_mode=True))

@main = primfn(A_1: handle, B_1: handle, C_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {A: Buffer(A_2: Pointer(float32), float32, [2097152], []),
             B: Buffer(B_2: Pointer(float32), float32, [1048576], []),
             C: Buffer(C_2: Pointer(float32), float32, [524288], [])}
  buffer_map = {A_1: A, B_1: B, C_1: C} {
  attr [IterVar(blockIdx.x: int32, (nullptr), "ThreadIndex", "blockIdx.x")] "thread_extent" = 128;
  attr [IterVar(threadIdx.x: int32, (nullptr), "ThreadIndex", "threadIdx.x")] "thread_extent" = 8;
  attr [IterVar(blockIdx.y: int32, (nullptr), "ThreadIndex", "blockIdx.y")] "thread_extent" = 64;
  attr [IterVar(threadIdx.y: int32, (nullptr), "ThreadIndex", "threadIdx.y")] "thread_extent" = 8 {
    C[((((blockIdx.x*4096) + (threadIdx.x*512)) + (blockIdx.y*8)) + threadIdx.y)] = 0f32
    for (k: int32, 0, 2048) {
      C[((((blockIdx.x*4096) + (threadIdx.x*512)) + (blockIdx.y*8)) + threadIdx.y)

In [27]:
%cd {PROJECT_ROOT}
!python -m pytest tests/test_gemm_gpu.py

/content/gdrive/MyDrive/ece5545/a3-ethanglaser
============================= test session starts ==============================
platform linux -- Python 3.7.13, pytest-3.6.4, py-1.11.0, pluggy-0.7.1
rootdir: /content/gdrive/MyDrive/ece5545/a3-ethanglaser, inifile:
plugins: typeguard-2.7.1
collected 20 items                                                             

tests/test_gemm_gpu.py ....................                              [100%]

========================== 20 passed in 10.97 seconds ==========================
